## Imports.

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
import re
import json
import rdata
import torch
import random
import pickle
import datasets
import numpy as np
import transformers
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from typing import Optional
import statsmodels.api as sm
from numpy.linalg import norm
from llm_measure import measure
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
from  datasets import load_dataset
from readability import Readability
from sklearn.decomposition import PCA
from nltk.tokenize import sent_tokenize
from scipy.stats.mstats import winsorize
from dataclasses import dataclass, field
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

## Review data.

In [5]:
with open('./data/review_total.pkl', 'rb') as file: 
    total_df = pickle.load(file) 
all_texts = total_df['review'].tolist()
probing_texts = total_df['review'].tolist()

## OLS.

In [6]:
def ols(df,
        out_name, 
        model_name, 
        probing_size,
        lda=False,
       ):
    with open('./results/' + out_name + f"_{model_name}" + f"_probingSize-{probing_size}" + '_pc.pkl', 'rb') as file: 
         pc = pickle.load(file) 
    with open('./results/' + out_name + f"_{model_name}" + f"_probingSize-{probing_size}" + '_scale.pkl', 'rb') as file: 
         scaler = pickle.load(file)
    with open('./results/' + out_name + f"_{model_name}" + f"_probingSize-{probing_size}" + '_emd.pkl', 'rb') as file:
         embeddings = pickle.load(file)
            
    direction = -pc.components_[0,:]
    our_llm_measure = np.dot(embeddings, direction)
    our_llm_measure_for_return = our_llm_measure.copy()
    
    scaler_2 = StandardScaler()
    our_llm_measure = scaler_2.fit_transform(our_llm_measure.reshape(-1, 1))
    our_llm_measure = our_llm_measure.reshape(-1)    

    regression_df = df.copy()
    regression_df['ours'] = our_llm_measure

    ####################### LLM-Measure ########################
    var_x = regression_df[['log_num_words', 'ours', 'binary_rating']]
    var_x = sm.add_constant(var_x)
    var_y = regression_df["log_helpfulness"]
    
    result = sm.OLS(var_y, var_x).fit()
    print(result.summary())
    
    
    ####################### LDA ########################    
    topicEntropy_LDA_standardized_for_return = regression_df['topicEntropy_LDA'].copy()   
    scaler_2 = StandardScaler()
    topicEntropy_LDA_standardized = scaler_2.fit_transform(np.array(regression_df['topicEntropy_LDA']).reshape(-1, 1))
    topicEntropy_LDA_standardized = topicEntropy_LDA_standardized.reshape(-1) 
    
    regression_df['topicEntropy_LDA_standardized'] = topicEntropy_LDA_standardized
    var_x = regression_df[['log_num_words', 'topicEntropy_LDA_standardized', 'binary_rating']]
    var_x = sm.add_constant(var_x)
    var_y = regression_df["log_helpfulness"]
    result = sm.OLS(var_y, var_x).fit()
    if lda:
        print(result.summary())   
    return our_llm_measure_for_return, topicEntropy_LDA_standardized_for_return, direction

## Result 1 (main).

In [7]:
_, _, _ = ols(df=total_df,
             out_name = "original-signed",
             model_name = "Llama-2-13b-chat-hf",
             probing_size = 64,
             lda = True,
            )

                            OLS Regression Results                            
Dep. Variable:        log_helpfulness   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     232.4
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          9.39e-146
Time:                        19:07:09   Log-Likelihood:                -9771.7
No. Observations:               10000   AIC:                         1.955e+04
Df Residuals:                    9996   BIC:                         1.958e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4538      0.050     -9.078

## Result 2 (definition rephrased).

In [8]:
_, _, _= ols(df=total_df,
             out_name="definitionRephrased-signed",
             model_name="Llama-2-13b-chat-hf",
             probing_size=64,
            )

                            OLS Regression Results                            
Dep. Variable:        log_helpfulness   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     232.1
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          1.48e-145
Time:                        19:07:11   Log-Likelihood:                -9772.1
No. Observations:               10000   AIC:                         1.955e+04
Df Residuals:                    9996   BIC:                         1.958e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4302      0.048     -8.887

## Result 3 (instruction rephrased).

In [9]:
_, _, _ = ols(df=total_df, 
              out_name="instructionRephrased-signed",
              model_name="Llama-2-13b-chat-hf",
              probing_size=64,
             )

                            OLS Regression Results                            
Dep. Variable:        log_helpfulness   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     232.9
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          4.32e-146
Time:                        19:07:12   Log-Likelihood:                -9770.9
No. Observations:               10000   AIC:                         1.955e+04
Df Residuals:                    9996   BIC:                         1.958e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4448      0.049     -9.055

## Result 4 (probing size 128).

In [10]:
_,  _, _ = ols(df=total_df, 
                    out_name="original-signed",
                    model_name="Llama-2-13b-chat-hf",
                    probing_size=128,
                   )

                            OLS Regression Results                            
Dep. Variable:        log_helpfulness   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     232.2
Date:                Mon, 09 Sep 2024   Prob (F-statistic):          1.19e-145
Time:                        19:07:13   Log-Likelihood:                -9771.9
No. Observations:               10000   AIC:                         1.955e+04
Df Residuals:                    9996   BIC:                         1.958e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4506      0.050     -9.045